# Нейронные сети с PyTorch

Сети глубокого обучения, как правило, имеют множество слоев, до десятков или даже сотен, и именно от этого происходит термин "глубокий". Вы можете построить одну из таких глубоких сетей, используя только матрицы весов, как мы делали в предыдущем блокноте, но, как правило, это очень неудобно и сложно для реализации. PyTorch имеет удобный модуль `nn`, который позволяет эффективно строить большие нейронные сети.

In [ ]:
# Импорт необходимых пакетов

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch

import helper

import matplotlib.pyplot as plt

Теперь мы собираемся построить более крупную сеть, которая сможет решить (ранее) сложную задачу по распознаванию текста на изображении. Будем использовать набор данных MNIST, который состоит из черно-белых рукописных цифр. Каждое изображение имеет размер 28x28 пикселей (см. примеры ниже).

<img src='assets/mnist.png'>

Наша цель — построить нейронную сеть, которая сможет взять одно из этих изображений и предсказать цифру на нем.

Сначала нужно получить этот набор данных. Он предоставляется через пакет `torchvision`. Код ниже загрузит набор данных MNIST, а затем создаст обучающие и тестовые наборы данных. Более детально этапы загрузки разберем в дальнейшем.

In [ ]:
# Наборы данных MNIST размещены на yann.lecun.com, который переместился под защиту CloudFlare
# Запустите этот скрипт, чтобы сделать возможной загрузку этого набора данных
# Ссылка: https://github.com/pytorch/vision/issues/1938

from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [ ]:
### Запустите эту ячейку

from torchvision import datasets, transforms

# Определим преобразование для нормализации данных
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
# Загрузим обучающие данные
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

Мы загрузили обучающие данные в `trainloader` и сделали итератор с помощью `iter(trainloader)`. Позже мы будем использовать этот объект для перебора набора данных для обучения, например,

```python
for image, label in trainloader:
    ## делаем что-то с изображениями и метками
```

Объект `trainloader` создан с размером пакета (batch size) 64 и `shuffle=True`. Размер пакета (batch size) — это количество изображений, которые мы получаем за одну итерацию из загрузчика данных и передаем через нашу сеть, набор изображений - *пакет* (batch). А `shuffle=True` указывает на то, чтобы перемешивать набор данных каждый раз, когда мы начинаем снова проходить через загрузчик данных. Здесь берется первый пакет, чтобы мы могли проверить данные. Мы можем увидеть ниже, что `images` — это просто тензор с размером `(64, 1, 28, 28)`. Итак, 64 изображения на пакет, 1 цветовой канал и изображения размером 28x28.

In [ ]:
dataiter = iter(trainloader)
images, labels = next(dataiter)
print(type(images))
print(images.shape)
print(labels.shape)

Вот как выглядит одно из изображений.

In [ ]:
plt.imshow(images[1].numpy().squeeze(), cmap='Greys_r')
plt.show()

Сначала давайте попытаемся построить простую сеть для этого набора данных, используя матрицы весов и матричные умножения. Увидим, как это сделать, используя модуль `nn` в PyTorch, который предоставляет удобный и мощный метод для определения архитектур сети.

Сети, которые вы видели до сих пор, называются *полносвязными* (fully-connected) или *плотными* (dense) сетями. Каждый узел в одном слое соединен с каждым узлом в следующем слое. В полносвязных сетях на вход каждого слоя должен поступать одномерный вектор (который можно сложить в 2D тензор как набор нескольких примеров). Однако наши изображения являются 2D тензорами размером 28x28, поэтому нужно преобразовать их в 1D векторы. Если говорить о размерах, нам нужно преобразовать пакет изображений с формой `(64, 1, 28, 28)` в форму `(64, 784)`, где 784 — это 28*28. Это обычно называется *разворачиванием* (flattening), мы разворачиваем 2D изображения в 1D векторы.

Ранее вы построили сеть с одной выходным узлом. Здесь нам нужно 10 выходных узлов, по одному для каждой цифры. Мы хотим, чтобы наша сеть предсказывала цифру, показанную на изображении, поэтому будем вычислять вероятности того, что изображение соответствует любой одной цифре или классу. Это в конечном счете представляет собой дискретное распределение вероятностей по классам (цифрам), которое говорит нам о наиболее вероятном классе для изображения. Это означает, что нам нужно 10 выходов для 10 классов (цифр). Мы увидим, как преобразовать выходные данные сети в распределение вероятностей дальше.

> **Упражнение:** Разверните пакет изображений `images`. Затем постройте многослойную сеть с 784 входными узлами, 256 скрытыми узлами и 10 выходными узлами, используя случайные тензоры для весов и смещений. Пока используйте сигмоидную активацию для скрытого слоя. Оставьте выходной слой без активации, мы добавим один слой, который даст нам распределение вероятностей в дальнейшем.

In [ ]:
## TODO


out = # выход вашей сети, должен иметь форму (64,10)

Теперь у нас есть 10 выходов для нашей сети. Мы хотим передать изображение в нашу сеть и получить распределение вероятностей по классам, которое скажет нам, к какому классу принадлежит изображение. Что-то вроде этого:
<img src='assets/image_distribution.png' width=500px>

Здесь мы видим, что вероятность для каждого класса примерно одинакова. Это картина соответствует необученной сети, она еще не видела данных, поэтому возвращает равномерное распределение с одинаковыми вероятностями для каждого класса.

Чтобы вычислить распределение вероятностей, мы часто используем [**softmax** функцию](https://en.wikipedia.org/wiki/Softmax_function). Математически эта функция выглядит как:

$$
\Large \sigma(x_i) = \cfrac{e^{x_i}}{\sum_k^K{e^{x_k}}}
$$

Эта функция сжимает каждое входное значение $x_i$ в диапазон от 0 до 1 и нормализует значения, чтобы дать вам правильное распределение вероятностей, где вероятности в сумме составляют единицу.

> **Упражнение:** Реализуйте функцию `softmax`, которая выполняет вычисление softmax и возвращает распределения вероятностей для каждого примера в пакете. Обратите внимание, что вам нужно будет уделить внимание размерностям при выполнении этого задания. Если у вас есть тензор `a` с формой `(64, 10)` и тензор `b` с формой `(64,)`, выполнение `a/b` приведет к ошибке, потому что PyTorch попытается выполнить деление по столбцам (broadcasting), но вы получите несоответствие размеров. Каждый из 64 примеров вы хотите делить только на одно значение, сумму в знаменателе. Поэтому вам нужно, чтобы `b` имел форму `(64, 1)`. Таким образом, PyTorch разделит 10 значений в каждой строке `a` на одно значение в каждой строке `b`. Также обратите внимание на то, как вы берете сумму. Вам нужно будет задать ключевое слово `dim` в `torch.sum`. Установка `dim=0` берет сумму по строкам, в то время как `dim=1` берет сумму по столбцам.

In [ ]:
## TODO
def softmax(x):
    # реализуйте функцию

# Переменная out здесь - выход вашей сети из прошлого упражнения с формой (64,10)
probabilities = softmax(out)

# Имеет ли probabilities правильную форму? Должно быть (64, 10)
print(probabilities.shape)
# Сумма равна 1?
print(probabilities.sum(dim=1))

## Построение сетей с PyTorch

PyTorch предоставляет модуль `nn`, который упрощает построение сетей. Покажем, как построить такую же сеть, как выше, с 784 входами, 256 скрытыми узлами, 10 выходными узлами и выходом softmax.

In [ ]:
from torch import nn

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Линейное преобразование от входов к скрытому слою
        self.hidden = nn.Linear(784, 256)
        # Выходной слой, 10 узлов - по одному для каждой цифры
        self.output = nn.Linear(256, 10)
        
        # Определим сигмоидную функцию активации и softmax выход
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        # Пропускаем входной тензор через каждую из наших операций
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.output(x)
        x = self.softmax(x)
        
        return x

Давайте разберем это по частям.

```python
class Network(nn.Module):
```

Здесь мы наследуемся от `nn.Module`. Вместе с `super().__init__()` это создает класс, который отслеживает архитектуру и предоставляет множество полезных методов и атрибутов. Наследование от `nn.Module` обязательно при создании класса для вашей сети. Имя самого класса может быть любым.

```python
self.hidden = nn.Linear(784, 256)
```

Эта строка создает модуль для линейного преобразования, $x\mathbf{W} + b$, с 784 входами и 256 выходами и присваивает его `self.hidden`. Модуль автоматически создает тензоры весов и смещений, которые мы будем использовать в методе `forward`. Вы можете получить доступ к тензорам весов и смещений после создания сети (`net`) с помощью `net.hidden.weight` и `net.hidden.bias`.

```python
self.output = nn.Linear(256, 10)
```

Аналогично, это создает другое линейное преобразование с 256 входами и 10 выходами.

```python
self.sigmoid = nn.Sigmoid()
self.softmax = nn.Softmax(dim=1)
```

Здесь определены операции для сигмоидальной активации и выхода softmax. Установка `dim=1` в `nn.Softmax(dim=1)` вычисляет softmax по столбцам.

```python
def forward(self, x):
```

Сети PyTorch, созданные с помощью `nn.Module`, должны иметь определенный метод `forward`. Он принимает вектор `x` и пропускает его через операции, которые вы определили в методе `__init__`.

```python
x = self.hidden(x)
x = self.sigmoid(x)
x = self.output(x)
x = self.softmax(x)
```

Здесь входной тензор `x` пропускается через каждую операцию и переназначается в `x`. Мы видим, что входной тензор проходит через скрытый слой, затем через сигмоидальную функцию, затем через выходной слой и, наконец, через функцию softmax. Неважно, как вы назвали переменные здесь, главное, чтобы входы и выходы операций соответствовали архитектуре сети, которую вы хотите построить. Порядок, в котором вы определяете элементы в методе `__init__`, не имеет значения, но вам нужно правильно упорядочить операции в методе `forward`.

Теперь мы можем создать объект `Network`.

In [ ]:
# Создадим сеть и посмотрим на ее текстовое представление
model = Network()
model

Вы можете определить сеть немного более лаконично и ясно, используя модуль `torch.nn.functional`. Обычно мы импортируем этот модуль как `F`, `import torch.nn.functional as F`.

In [ ]:
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # Линейное преобразование от входов к скрытому слою
        self.hidden = nn.Linear(784, 256)
        # Выходной слой, 10 выходов - по одному для каждой цифры
        self.output = nn.Linear(256, 10)
        
    def forward(self, x):
        # Скрытый слой с сигмоидной функцией активации
        x = F.sigmoid(self.hidden(x))
        # Выходной слой с активацией softmax
        x = F.softmax(self.output(x), dim=1)
        
        return x

### Функции активации

До сих пор мы смотрели только на сигмоиду и softmax в качестве функций активации, но в общем случае любую функцию можно использовать в качестве активационной функции. Единственное требование состоит в том, что для того чтобы сеть могла аппроксимировать нелинейную функцию, активационные функции должны быть нелинейными. Вот еще несколько примеров распространенных функций активации: Tanh (гиперболический тангенс), и ReLU (rectified linear unit).

<img src="assets/activation.png" width=700px>

На практике функция ReLU используется крайне часто в качестве функции активации для скрытых слоев.

### Постройте сеть самостоятельно

<img src="assets/mlp_mnist.png" width=600px>

> **Упражнение:** Создайте сеть с 784 входными узлами, скрытым слоем с 128 узлами и активацией ReLU, затем скрытым слоем с 64 узлами и активацией ReLU, и, наконец, выходным слоем с активацией softmax, как показано выше. Вы можете использовать активацию ReLU с модулем `nn.ReLU` или функцией `F.relu`.

Хорошей практикой является именование слоев по их типу, например, 'fc', чтобы обозначить полносвязный слой. Когда вы будете писать свое решение, используйте `fc1`, `fc2` и `fc3` в качестве имен ваших слоев.

In [ ]:
## TODO


### Инициализация весов и смещений

Веса и смещения автоматически инициализируются для вас, но вы можете настроить, как они инициализируются. Веса и смещения — это тензоры, связанные с определенным слоем, вы можете получить к ним доступ с помощью `model.fc1.weight` например.

In [ ]:
print(model.fc1.weight)
print(model.fc1.bias)

Для пользовательской инициализации мы хотим модифицировать эти тензоры на месте. На самом деле это *переменные* autograd, поэтому нам нужно вернуть настоящие тензоры с помощью `model.fc1.weight.data`. Как только у нас будут тензоры, мы можем заполнить их нулями (для смещений) или случайными нормальными значениями.

In [ ]:
# Установим смещения в нули
model.fc1.bias.data.fill_(0)

In [ ]:
# выборка из случайных значений из нормального распределения с стандартным отклонением = 0.01
model.fc1.weight.data.normal_(std=0.01)

### Прямой проход

Теперь, когда у нас есть сеть, давайте посмотрим, что произойдет, когда мы передадим изображение.

In [ ]:
# Возьмем некоторые данные 
dataiter = iter(trainloader)
images, labels = next(dataiter)

# Изменим размер изображений на 1D вектор, новая форма (размер пакета, цветовые каналы, пиксели изображения) 
images.resize_(64, 1, 784)
# или images.resize_(images.shape[0], 1, 784), чтобы автоматически получить размер пакета

# Прямой проход через сеть
img_idx = 0
ps = model.forward(images[img_idx,:])

img = images[img_idx]
helper.view_classify(img.view(1, 28, 28), ps)

Как вы можете видеть выше, наша сеть не имеет понятия о том, какая цифра. Это потому, что мы еще не обучили ее, все веса случайны.

### Использование `nn.Sequential`

PyTorch предоставляет удобный способ строить сети, где тензор последовательно проходит через операции, `nn.Sequential` ([документация](https://pytorch.org/docs/master/nn.html#torch.nn.Sequential)). Применим этот способ, чтобы построить эквивалентную сеть:

In [ ]:
# Гиперпараметры для нашей сети
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

# Построим сеть
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.Softmax(dim=1))
print(model)

# Прямой проход через сеть и вывод результата
images, labels = next(iter(trainloader))
images.resize_(images.shape[0], 1, 784)
ps = model.forward(images[0,:])
helper.view_classify(images[0].view(1, 28, 28), ps)

Операции доступны по соответствующему индексу. Например, если вы хотите получить первую линейную операцию и взглянуть на веса, вы можете использовать `model[0]`.

In [ ]:
print(model[0])
model[0].weight

Вы также можете передать `OrderedDict`, чтобы назвать отдельные слои и операции, вместо того, чтобы использовать последовательные целые числа. Обратите внимание, что ключи словаря должны быть уникальными, поэтому _каждая операция должна иметь другое имя_.

In [ ]:
from collections import OrderedDict
model = nn.Sequential(OrderedDict([
                      ('fc1', nn.Linear(input_size, hidden_sizes[0])),
                      ('relu1', nn.ReLU()),
                      ('fc2', nn.Linear(hidden_sizes[0], hidden_sizes[1])),
                      ('relu2', nn.ReLU()),
                      ('output', nn.Linear(hidden_sizes[1], output_size)),
                      ('softmax', nn.Softmax(dim=1))]))
model

Теперь вы можете получить доступ к слоям как по индексу, так и по названию.

In [ ]:
print(model[0])
print(model.fc1)

В следующем блокноте мы увидим, как мы можем обучить нейронную сеть для точного предсказания цифр на изображениях MNIST.